Prerequisite
===========================================

In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Data preparation<br>


In [ ]:
!git clone https://github.com/apache/spark.git

In [ ]:
%cd spark/
!git log --date=short --pretty=format:'%h,%an,%ad,%s' >> ./commitlog.csv

In [ ]:
!cat commitlog.csv

Word Count (Part One)
===========================

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
df = spark.read.csv('commitlog.csv')
df.show(1)

+----------+---------+----------+--------------------+
|       _c0|      _c1|       _c2|                 _c3|
+----------+---------+----------+--------------------+
|89e907f76c|Sean Owen|2021-08-29|[SPARK-36603][COR...|
+----------+---------+----------+--------------------+
only showing top 1 row



In [ ]:
renamed_df = df.withColumnRenamed('_c0','hash') \
.withColumnRenamed('_c1','contributor') \
.withColumnRenamed('_c2','dt') \
.withColumnRenamed('_c3','message')
renamed_df.show(1)

+----------+-----------+----------+--------------------+
|      hash|contributor|        dt|             message|
+----------+-----------+----------+--------------------+
|89e907f76c|  Sean Owen|2021-08-29|[SPARK-36603][COR...|
+----------+-----------+----------+--------------------+
only showing top 1 row



In [ ]:
from pyspark.sql.functions import *
words = renamed_df.select('hash', \
'contributor', \
'dt', \
explode(split(col('message'),' ')).alias('word'))

In [ ]:
words.filter('hash="281b00ab5b"').show()

+----------+--------------+----------+--------------------+
|      hash|   contributor|        dt|                word|
+----------+--------------+----------+--------------------+
|281b00ab5b|Kousuke Saruta|2021-08-18|[SPARK-34309][BUI...|
|281b00ab5b|Kousuke Saruta|2021-08-18|             Upgrade|
|281b00ab5b|Kousuke Saruta|2021-08-18|            Caffeine|
|281b00ab5b|Kousuke Saruta|2021-08-18|                  to|
|281b00ab5b|Kousuke Saruta|2021-08-18|               2.9.2|
+----------+--------------+----------+--------------------+



In [ ]:
words.groupBy(lower(col('word')).alias('word')) \
.count() \
.orderBy(desc(col('count'))) \
.show()

+-------+-----+
|   word|count|
+-------+-----+
|     in| 7354|
|     to| 6838|
|    for| 5407|
|    the| 3970|
|    fix| 3462|
|     of| 3413|
|    and| 3162|
|    add| 3152|
|   from| 2340|
|      a| 1875|
|  merge| 1755|
|support| 1553|
|   with| 1536|
|   when| 1485|
|    use| 1455|
| should| 1424|
|  [sql]| 1364|
| remove| 1226|
|   pull| 1190|
|     on| 1174|
+-------+-----+
only showing top 20 rows



# Word Count (Part Two)


In [ ]:
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [ ]:
words.groupBy(lower(col('word')).alias('word')) \
.count() \
.orderBy(desc(col('count'))) \
.filter(~col('word').isin(stop_words)).show()

+-------+-----+
|   word|count|
+-------+-----+
|    fix| 3462|
|    add| 3152|
|  merge| 1755|
|support| 1553|
|    use| 1455|
|  [sql]| 1364|
| remove| 1226|
|   pull| 1190|
|request| 1150|
|   test| 1122|
| update|  913|
|  spark|  860|
|   make|  764|
|    api|  725|
| python|  698|
|  table|  676|
|    sql|  673|
|  tests|  663|
|   code|  660|
|   data|  627|
+-------+-----+
only showing top 20 rows



In [ ]:
words.groupBy('contributor',lower(col('word')).alias('word')) \
.count() \
.orderBy(desc(col('count'))) \
.filter(~col('word').isin(stop_words)).show()

+---------------+---------+-----+
|    contributor|     word|count|
+---------------+---------+-----+
|  Matei Zaharia|    merge|  764|
|  Matei Zaharia|     pull|  612|
|  Matei Zaharia|  request|  610|
|    Reynold Xin|    merge|  269|
|     Davies Liu|    [sql]|  235|
|    Reynold Xin|     pull|  215|
|    Reynold Xin|  request|  208|
|Patrick Wendell|     pull|  200|
|Patrick Wendell|    merge|  184|
|Patrick Wendell|  request|  164|
|  Matei Zaharia|   branch|  152|
|     Cheng Lian|    [sql]|  142|
|     Davies Liu|[pyspark]|  138|
|  Dongjoon Hyun|      fix|  122|
|    Reynold Xin|   remove|  122|
|  Dongjoon Hyun|      add|  116|
|     Josh Rosen|      fix|  112|
|  Matei Zaharia|      fix|  111|
|     Davies Liu|      fix|  110|
|   Shixiong Zhu|      fix|   99|
+---------------+---------+-----+
only showing top 20 rows



In [ ]:
stop_words += ["merge", "pull", "request", "fix"]

words \
.filter(col('dt')>'2020-12-31') \
.groupBy(lower(col('word')).alias('word')) \
.count()\
.orderBy(desc(col('count'))) \
.filter(~col('word') \
.isin(stop_words)) \
.show()

+---------+-----+
|     word|count|
+---------+-----+
|      add|  244|
|  support|  182|
|      use|  112|
|   remove|   91|
|     type|   87|
|     test|   85|
|  upgrade|   81|
|    table|   76|
|   update|   73|
|    check|   72|
| interval|   71|
|      sql|   60|
|     ansi|   58|
|     data|   57|
|    tests|   51|
|    avoid|   50|
|    scala|   49|
| function|   49|
|     make|   49|
|exception|   47|
+---------+-----+
only showing top 20 rows



In [ ]:
words \
.filter(col('dt')>'2020-12-31') \
.filter(lower(col('word')).contains('py')) \
.count()

399

In [ ]:
words \
.filter(col('dt')>'2020-12-31') \
.filter(lower(col('word')).contains('scala')) \
.count()

86